In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import catboost as cgb
from bayes_opt import BayesianOptimization
import warnings
import wandb
import time
warnings.filterwarnings('ignore')

In [8]:
METHOD = 'bayes'

In [18]:
cat_features=[i for i in X.columns if X.dtypes[i]!='float']

In [3]:
data = pd.read_csv('house_prices_clean.csv')

In [4]:
data.shape

(1460, 127)

In [5]:
X = data.drop('SalePrice',axis =1)

y = data['SalePrice']

In [19]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X,y,test_size=0.2, random_state=123)

In [7]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    Calculates Root Mean Squared Log Error between predictions and true labels
    """
    return np.sqrt(mean_squared_log_error(y_test,y_preds))

def rmse(predictions, targets): 

    return np.sqrt(((predictions - targets) ** 2).mean())

def rmsle2(y, y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

# Create function to evaluate model on a few different levels
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_test)
    scores = {'Training MAE': mean_absolute_error(y_train,train_preds),
             'Valid MAE': mean_absolute_error(y_test,val_preds),
             'Training R^2': r2_score(y_train,train_preds),
             'Valid R^2': r2_score(y_test,val_preds),
             'Training RMSE': rmse(y_train,train_preds),
             'Valid RMSE': rmse(y_test,val_preds),
             'Training RMSLE2': rmsle2(y_train,train_preds),
             'Valid RMSLE2': rmsle2(y_test,val_preds)
             }
    
    return scores
    

In [28]:
def train():
    # Prepare data tuples
    (X_train, y_train) = X_train1, y_train1
    (X_test, y_test) = X_test1, y_test1
    
    # Default values for hyper-parameters we're going to sweep over
    configs = {
            'num_leaves': 100,            
            'feature_fraction': 0.1,
            'bagging_fraction': 0.1,
            'max_depth': 200,
            'min_split_gain': 0.001,
            'min_child_weight': 1,
            'method': METHOD
    }

    # Initilize a new wandb run
    wandb.init(project='hyperparameter-sweeps-lgb', config=configs)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Define the model
    clf = lgb.LGBMRegressor(bagging_fraction = config.bagging_fraction,
                       feature_fraction = config.feature_fraction,
                       max_depth = config.max_depth,
                       min_child_weight = config.min_child_weight,
                       min_split_gain = config.min_split_gain,
                       num_leaves = config.num_leaves)
    
    # Train the model
    clf.fit(X_train,y_train)
    


In [30]:
def get_sweep_id(method):
    sweep_config = {
        'method': method,
        'metric': {
          'name': 'rmse',
          'goal': 'minimize'   
        },
        'parameters':{
            'num_leaves':{
                'values': (100, 700)
            },
            'feature_fraction':{ 'values': (0.1, 0.9)},
            'bagging_fraction':{ 'values': (0.1, 1)},
            'max_depth':{ 'values': (200, 500)},
            'min_split_gain':{ 'values': (0.001, 0.1)},
            'min_child_weight': {'values': (1, 100) }
            }
        }
    
    sweep_id = wandb.sweep(sweep_config, project='hyperparameter-sweeps-lgb')
    
    return sweep_id

In [26]:
sweep_id = get_sweep_id('bayes')

Create sweep with ID: 6j6xqns5
Sweep URL: https://app.wandb.ai/sinisha123/hyperparameter-sweeps-comparison/sweeps/6j6xqns5


In [29]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: iftdiomt with config:
	bagging_fraction: 0.1
	feature_fraction: 0.9
	max_depth: 200
	min_child_weight: 100
	min_split_gain: 0.001
	num_leaves: 100
wandb: Agent Started Run: iftdiomt


wandb: WARNING Ignoring project='hyperparameter-sweeps-lgb' passed to wandb.init when running a sweep


wandb: Agent Finished Run: iftdiomt 

wandb: Agent Starting Run: xa5i0a6p with config:
	bagging_fraction: 0.1
	feature_fraction: 0.9
	max_depth: 500
	min_child_weight: 1
	min_split_gain: 0.001
	num_leaves: 100
wandb: Agent Started Run: xa5i0a6p


wandb: WARNING Ignoring project='hyperparameter-sweeps-lgb' passed to wandb.init when running a sweep


wandb: Agent Finished Run: xa5i0a6p 

wandb: Agent Starting Run: a3hbbf82 with config:
	bagging_fraction: 0.1
	feature_fraction: 0.9
	max_depth: 200
	min_child_weight: 1
	min_split_gain: 0.001
	num_leaves: 100
wandb: Agent Started Run: a3hbbf82


...

wandb: WARNING Ignoring project='hyperparameter-sweeps-lgb' passed to wandb.init when running a sweep


wandb: Agent Finished Run: rn9l610a 

wandb: Agent Starting Run: 09280wr1 with config:
	bagging_fraction: 0.1
	feature_fraction: 0.9
	max_depth: 200
	min_child_weight: 100
	min_split_gain: 0.001
	num_leaves: 100
wandb: Agent Started Run: 09280wr1


wandb: WARNING Ignoring project='hyperparameter-sweeps-lgb' passed to wandb.init when running a sweep


wandb: Agent Finished Run: 09280wr1 



wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.
Process Process-12:
Traceback (most recent call last):
  File "/home/sinisha/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/sinisha/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/wandb/wandb_agent.py", line 71, in _start
    run._stop_jupyter_agent()
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/wandb/wandb_run.py", line 182, in _stop_jupyter_agent
    self._jupyter_agent.stop()
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/wandb/jupyter.py", line 136, in stop
    self.rm.shutdown()
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/wandb/run_manager.py", line 1071, in shutdown
    self._end_file_syncing(exitcode)
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/w